In [2]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image as im
import matplotlib.image as mpimg
from tqdm.notebook import tqdm
from glob import glob
import os
import cv2
from PIL import Image
import pandas as pd
import shutil
from joblib import Parallel, delayed
ff = 10000000
f = 100



In [3]:

test_tar = "/home/smoothjazzuser/Desktop/TEST/"
train_tar = "/home/smoothjazzuser/Desktop/NORMAL-TRAIN"
test_tar = sorted(glob (test_tar + "/*/*/*.tar"), key = lambda x: int(x.split("-")[-1].split(".")[0]))

train_tar = sorted(glob(train_tar + "/*/*.tar"), key = lambda x: int(x.split("-")[-1].split(".")[0]))
train_tar = pd.DataFrame(train_tar)
test_tar = pd.DataFrame(test_tar)

if len(train_tar) > 0:
    train_tar['dest'] = train_tar[0].apply(lambda x: x[0:-11])
    test_tar['dest'] = test_tar[0].apply(lambda x: x[0:-11])

    train_tar['dest'][0]


In [6]:
#untar the files
def untar (tar_file, dest):
    os.system("tar -xf " + tar_file + " -C " + dest)
    return 1

#untar the files
_ = Parallel(n_jobs = 8)(delayed(untar)(tar_file, dest) for tar_file, dest in tqdm(zip(train_tar[0], train_tar['dest'])))
_ = Parallel(n_jobs = 8)(delayed(untar)(tar_file, dest) for tar_file, dest in tqdm(zip(test_tar[0], test_tar['dest'])))

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [7]:
# delete the tar files
def delete_tar (tar_file):
    os.system("rm " + tar_file)
    return 1

Parallel(n_jobs = 8)(delayed(delete_tar)(tar_file) for tar_file in tqdm(train_tar[0]))
Parallel(n_jobs = 8)(delayed(delete_tar)(tar_file) for tar_file in tqdm(test_tar[0]))

  0%|          | 0/60 [00:00<?, ?it/s]

  0%|          | 0/119 [00:00<?, ?it/s]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1]

In [10]:


def vis(img): #reorders array to be compatible with matplotlib
    im = np.moveaxis(img,0, -1)
    plt.imshow(im)
    plt.show()


def numpy_to_image(img, path): #converts numpy array to image
    if not os.path.exists(f'{path}.png'):
        img = np.moveaxis(img,0, -1)
        #img = np.dot(img[...,:3], [0.299, 0.587, 0.114]) 
        mpimg.imsave(f'{path}.png', img)#, cmap='gray')
        #Image.fromarray(img).save(f'{path}.png')
        return 0,0
    else:
        img = np.moveaxis(img,0, -1)
        print(f'{path}.png')
        b="/".join(path.split('/')[0:-2])
        bb = int(path.split('/')[-1])+1
        #img = np.dot(img[...,:3], [0.299, 0.587, 0.114]) 
        mpimg.imsave(f"""{b}{bb}.png""", img)#,cmap='gray')
        #Image.fromarray(img).save(f"""{b}{bb}.png""")
        return 1,0
def binarize(i):
    return 1 if i > 0 else 0

# name = test_mask(f, False)
#print(name)

test_data = np.load(f'/home/smoothjazzuser/Desktop/NORMAL-TRAIN/ep-0000/00000000.npz', allow_pickle=True)
test_data.files
test_data['state'].shape
tr = "/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames/"
te = "/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/"
#
#glob('/home/smoothjazzuser/Desktop/fast/TEST/*')
#glob('/home/smoothjazzuser/Desktop/fast/TEST/BlackScreen/*/*.npz')

In [11]:
#make sure each dataset contains multiples of five images
def check_dataset(path):
    for i in glob(path + '/*'):
        files = glob(i + '/*')
        num = len(files)
        div = num/5
        if div % 1 != 0:
            print(i, num, div)
        

check_dataset(tr)
check_dataset(te)

In [12]:
#sorted(glob.glob(f"/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames/54/*.png"), key=lambda x: int(x.split('/')[-1].split('.')[0]))

masked_list = []
folder_count = 0
# for every file in each bug type folder, save the image to the corresponding folder under MNAD
with tqdm(total=len(glob('/home/smoothjazzuser/Desktop/TEST/*'))) as t:
    for i in glob('/home/smoothjazzuser/Desktop/TEST/*'):
        
        ff = glob(f'{i}/*')
        ff.sort()

        for ii in ff:
            
            counter = 0
            folder_count+=1
            if len(str(folder_count)) >= 2:
                    f_name = '' + str(folder_count)
            else:
                f_name = '0' + str(folder_count)

            images = glob(f'{ii}/*.npz')
            images.sort()

            for j in images:
                
                if len(str(counter)) >= 3:
                    name = str(counter)
                elif len(str(counter)) == 2:
                    name = '0' + str(counter)
                else:
                    name = '00' + str(counter)
                
                data = np.load(j, allow_pickle=True)
                masked_list.append(binarize(np.sum(data['info'][1])))
                if not os.path.exists(f"{te + f_name}"):
                    os.makedirs(f"{te + f_name}")
                c, f =numpy_to_image(data['state'], te + f_name + "/" + name)
                counter += c
                folder_count += f
                counter+=1
            
    t.update(1)

np.save('frame_labels_bugs.npy', np.array(np.expand_dims(np.array(masked_list), axis=0)))

  0%|          | 0/10 [00:00<?, ?it/s]

In [13]:
del masked_list
import gc
gc.collect()

227

In [14]:
folder_count = 0
for i in tqdm(glob('/home/smoothjazzuser/Desktop/NORMAL-TRAIN/*')):

    folder_count+=1
    if len(str(folder_count)) >= 2:
        f_name = '' + str(folder_count)
    else:
        f_name = '0' + str(folder_count)
    counter = -1
    if not os.path.exists(f"{tr + f_name}"):
            os.makedirs(f"{tr + f_name}")
    
    images = glob(f'{i}/*.npz')
    images.sort()

    for j in images:
        counter+=1
        if len(str(counter)) >= 3:
            name = str(counter)
        elif len(str(counter)) == 2:
            name = '0' + str(counter)
        else:
            name = '00' + str(counter)
                
        data = np.load(j, allow_pickle=True)
        #masked_list.append(np.sum(data['info'][1]))
        numpy_to_image(data['state'], tr + f_name + "/" + name)
        
    

  0%|          | 0/61 [00:00<?, ?it/s]

In [15]:
if not os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GTall/'):
    os.makedirs('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GTall/')

In [16]:
# move frame_labels_bugs.npy to MNAD/dataset/frame_labels_bugs.npy

if not os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GTall/frame_labels_bugs.npy'):
    shutil.copy('/home/smoothjazzuser/videogame-anomoly/frame_labels_bugs.npy', '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GTall/frame_labels_bugs.npy')

In [17]:
#rename frames folder to full
if not os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/full'):
    os.rename('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/frames', '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/training/full')

In [18]:
#move files from /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames t/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc
if not os.path.exists('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/01'):
    shutil.move('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/testing/frames', '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc')
    #move files from symlink_loc/frames to symlink_loc
    for i in glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/frames/*'):
        shutil.move(i, '/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc')

In [26]:
# create folder structure for testing on the entire dataset
all_files_in_symlink_loc = glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/*/*.png') +  glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/*/*.jpg')
# sort by folder name and then by file integer name
all_files_in_symlink_loc = sorted(all_files_in_symlink_loc, key=lambda x: (x.split('/')[-2], int(x.split('/')[-1].split('.')[0])))
all_files_in_symlink_loc
if not os.path.exists("/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/all_testing"):
    os.makedirs("/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/all_testing")
if not os.path.exists("/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/all_testing/1"):
    os.makedirs("/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/all_testing/1")

In [29]:
# create symlink to all_testing/1 for all files in all_files_in_symlink_loc
for i,p in enumerate(all_files_in_symlink_loc):
    k = str(i).zfill(3)
    os.symlink(p, f"/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/all_testing/1/{k}.png")

In [30]:
# move /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/1000 to /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/1000
if not os.path.exists("/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/1000"):
    shutil.move("/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/1000", "/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc")
#copy GTall to say GT1000    
if not os.path.exists("/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT1000"):
    shutil.copytree("/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GTall", "/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT1000")

In [44]:
# list all files in "/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/1000"
all_files_in_GT1000 = glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/1000/*.png') +  glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/1000/*.jpg')
# sort by filename
all_files_in_1000 = sorted(all_files_in_GT1000, key=lambda x: int(x.split('/')[-1].split('.')[0]))
len(all_files_in_1000)

beg = len(all_files_in_1000) - int(len(all_files_in_1000)*.1)
end = len(all_files_in_1000)
# load GTall/frame_labels_bugs.npy
frame_labels_bugs = np.load('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GTall/frame_labels_bugs.npy')

# cut off last 10% of files in 1000 and move to 1001
if not os.path.exists("/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/1001"):
    os.makedirs("/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/1001")
    for i in range(beg, end):
        shutil.copy(all_files_in_1000[i], "/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/1001", follow_symlinks=True)

# move 10% of GT1000 to GT1001
if not os.path.exists("/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT1001"):
    os.makedirs("/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT1001")
    frame_labels_bugs_1001 = np.expand_dims(frame_labels_bugs[0][beg:end], axis=0)
    np.save('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT1001/frame_labels_bugs.npy', frame_labels_bugs_1001)

In [45]:
# rename all symlinks in /home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT1001/frame_labels_bugs.npy" starting from 000.png
all_files_in_GT1001 = glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT1001/*.png') +  glob('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT1001/*.jpg')
all_files_in_GT1001 = sorted(all_files_in_GT1001, key=lambda x: int(x.split('/')[-1].split('.')[0]))
for i,p in enumerate(all_files_in_GT1001):
    k = str(i).zfill(3)
    os.rename(p, f"/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT1001/{k}.png")

In [46]:
# read in GT1001/frame_labels_bugs.npy
frame_labels_bugs_1001 = np.load('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT1001/frame_labels_bugs.npy')
# read in GT1000/frame_labels_bugs.npy
frame_labels_bugs_1001.shape

(1, 49272)

In [47]:
# read in GT1001/frame_labels_bugs.npy
frame_labels_bugs_1000 = np.load('/home/smoothjazzuser/videogame-anomoly/MNAD/dataset/bugs/symlink_loc/GT1000/frame_labels_bugs.npy')
# read in GT1000/frame_labels_bugs.npy
frame_labels_bugs_1000.shape

(1, 492723)